In [75]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA


In [76]:
df = pd.read_csv(r"D:\I5-AMS\EDA\Project3\diamonds.csv")
df.head(5)

,Unnamed: 0,carat,cut,color,clarity,depth,table,price,x,y,z
0,1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


# **Data Preprocessing Step**

In [77]:
df = df.drop(columns=["Unnamed: 0"])

In [78]:
num_cols = ["carat", "depth", "table", "price", "x", "y", "z"]
for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

In [79]:
missing_table = (
    df.isna()
      .sum()
      .to_frame("count")   # column name like your screenshot
)

missing_table


,count
carat,0
cut,0
color,0
clarity,0
depth,0
table,0
price,0
x,0
y,0
z,0


In [80]:
missing_table = df.isna().sum().reset_index()
missing_table.columns = ["Column", "Missng Value"]
missing_table.style.hide(axis="index")


Column,Missng Value
carat,0
cut,0
color,0
clarity,0
depth,0
table,0
price,0
x,0
y,0
z,0


### Handling missing value

In [81]:
bad_summary = {
    "x == 0": (df["x"] == 0).sum(),
    "y == 0": (df["y"] == 0).sum(),
    "z == 0": (df["z"] == 0).sum(),
    "carat <= 0": (df["carat"] <= 0).sum(),
    "price <= 0": (df["price"] <= 0).sum(),
    "depth <= 0": (df["depth"] <= 0).sum(),
    "table <= 0": (df["table"] <= 0).sum(),
}

pd.DataFrame.from_dict(bad_summary, orient="index", columns=["count"])


,count
x == 0,8
y == 0,7
z == 0,20
carat <= 0,0
price <= 0,0
depth <= 0,0
table <= 0,0


In [82]:
bad_mask = (
    (df["x"] == 0) |
    (df["y"] == 0) |
    (df["z"] == 0) |
    (df["carat"] <= 0) |
    (df["price"] <= 0)
)

print("Total bad rows:", bad_mask.sum())

Total bad rows: 20


In [83]:
df.loc[bad_mask].head(20)

,carat,cut,color,clarity,depth,table,price,x,y,z
2207,1.00,Premium,G,SI2,59.1,59.0,3142,6.55,6.48,0.0
2314,1.01,Premium,H,I1,58.1,59.0,3167,6.66,6.60,0.0
4791,1.10,Premium,G,SI2,63.0,59.0,3696,6.50,6.47,0.0
5471,1.01,Premium,F,SI2,59.2,58.0,3837,6.50,6.47,0.0
10167,1.50,Good,G,I1,64.0,61.0,4731,7.15,7.04,0.0
11182,1.07,Ideal,F,SI2,61.6,56.0,4954,0.00,6.62,0.0
11963,1.00,Very Good,H,VS2,63.3,53.0,5139,0.00,0.00,0.0
13601,1.15,Ideal,G,VS2,59.2,56.0,5564,6.88,6.83,0.0
15951,1.14,Fair,G,VS1,57.5,67.0,6381,0.00,0.00,0.0
24394,2.18,Premium,H,SI2,59.4,61.0,12631,8.49,8.45,0.0


--> **Since the x, y and z is the measure of diamond, so 0 will not never exist on earth, we reegard it as missing value. So we need to handle it.** 

In [84]:
bad_xyz = (df["x"] == 0) | (df["y"] == 0) | (df["z"] == 0)

df_before = df.copy()
df_after = df.loc[~bad_xyz].copy()

print("Before shape:", df_before.shape)
print("After shape :", df_after.shape)
print("Dropped rows:", bad_xyz.sum())
print("Dropped %   :", bad_xyz.mean() * 100)


Before shape: (53940, 10)
After shape : (53920, 10)
Dropped rows: 20
Dropped %   : 0.03707823507601038


--> **The missing value is very small, we try to drop it out first.**

In [85]:
df_after.duplicated().sum()

np.int64(145)

In [86]:
df_after = df_after.drop_duplicates()

In [87]:
df = df_after.copy()

In [88]:
df.shape

(53775, 10)

In [89]:
df.dtypes

carat      float64
cut         object
color       object
clarity     object
depth      float64
table      float64
price        int64
x          float64
y          float64
z          float64
dtype: object

## Outlier

In [90]:
df["log_price"] = np.log1p(df["price"])

# **Preparing for PCA**

In [91]:
quan_cols = ["carat", "x", "y", "z", "depth", "table"]
X = df[quan_cols].dropna()
X.head()

,carat,x,y,z,depth,table
0,0.23,3.95,3.98,2.43,61.5,55.0
1,0.21,3.89,3.84,2.31,59.8,61.0
2,0.23,4.05,4.07,2.31,56.9,65.0
3,0.29,4.20,4.23,2.63,62.4,58.0
4,0.31,4.34,4.35,2.75,63.3,58.0


--> **Applying StandardScaler for getting the same interval**

In [92]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [93]:
pca_full = PCA()
scores_full = pca_full.fit_transform(X_scaled)
explained_var = pca_full.explained_variance_ratio_
cum_explained_var = np.cumsum(explained_var)
explained_var, cum_explained_var

(array([0.65716845, 0.21429729, 0.11360725, 0.00741234, 0.00557969,
        0.00193497]),
 array([0.65716845, 0.87146575, 0.985073  , 0.99248534, 0.99806503,
        1.        ]))

In [94]:
import plotly.graph_objects as go

pcs = [f"PC{i+1}" for i in range(len(explained_var))]

# --- Safe tab20c palette (works on ALL Plotly versions) ---
try:
    from plotly.colors import qualitative
    tab20c = qualitative.T20c  # may not exist on older plotly
except Exception:
    # Matplotlib tab20c hex palette (20 colors) - reliable fallback
    tab20c = [
        "#3182bd", "#6baed6", "#9ecae1", "#c6dbef",
        "#e6550d", "#fd8d3c", "#fdae6b", "#fdd0a2",
        "#31a354", "#74c476", "#a1d99b", "#c7e9c0",
        "#756bb1", "#9e9ac8", "#bcbddc", "#dadaeb",
        "#636363", "#969696", "#bdbdbd", "#d9d9d9"
    ]

bar_colors = [tab20c[i % len(tab20c)] for i in range(len(pcs))]

fig = go.Figure()

# Bars: explained variance
fig.add_trace(go.Bar(
    x=pcs,
    y=explained_var,
    name="Explained Variance",
    marker=dict(color=bar_colors, line=dict(width=0.6)),
    text=[f"{v:.1%}" for v in explained_var],
    textposition="outside",
    hovertemplate="<b>%{x}</b><br>Explained: %{y:.2%}<extra></extra>"
))

# Line: cumulative variance
fig.add_trace(go.Scatter(
    x=pcs,
    y=cum_explained_var,
    mode="lines+markers",
    name="Cumulative Variance",
    yaxis="y2",
    line=dict(width=3, color=tab20c[3]),
    marker=dict(size=8, color=tab20c[3]),
    hovertemplate="<b>%{x}</b><br>Cumulative: %{y:.2%}<extra></extra>"
))

# Thresholds on cumulative axis (80/90/95%)
for t in [0.80, 0.90, 0.95]:
    fig.add_hline(
        y=t,
        yref="y2",
        line_width=1,
        line_dash="dash",
        line_color="gray",
        opacity=0.7,
        annotation_text=f"{int(t*100)}%",
        annotation_position="top right"
    )

fig.update_layout(
    title="Scree Plot: Explained Variance & Cumulative Variance",
    template="plotly_white",
    xaxis=dict(title="Principal Components", tickangle=-30),
    yaxis=dict(title="Explained Variance Ratio", tickformat=".0%"),
    yaxis2=dict(
        title="Cumulative Explained Variance",
        overlaying="y",
        side="right",
        tickformat=".0%",
        range=[0, 1.05]
    ),
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="left", x=0),
    height=520,
    margin=dict(t=80, r=70, l=70, b=80)
)

fig.show()


In [95]:
pca = PCA(n_components=2, random_state=42)
scores = pca.fit_transform(X_scaled)

print("Explained variance PC1:", round(pca.explained_variance_ratio_[0]*100, 2), "%")
print("Explained variance PC2:", round(pca.explained_variance_ratio_[1]*100, 2), "%")
# print("Explained variance PC3:", round(pca.explained_variance_ratio_[2]*100, 2), "%")


Explained variance PC1: 65.72 %
Explained variance PC2: 21.43 %


In [96]:
# Corr(var, PCk) computed from standardized data and PC scores
Corr_matrix = pd.DataFrame(
    np.corrcoef(X_scaled.T, scores.T)[:len(quan_cols), len(quan_cols):],
    index=quan_cols,
    columns=["PC1", "PC2"]
)
Corr_matrix


,PC1,PC2
carat,0.984084,0.051444
x,0.993794,0.009321
y,0.982386,0.011001
z,0.982294,0.113961
depth,-0.001495,0.832129
table,0.238697,-0.759937


In [97]:
theta = np.linspace(0, 2*np.pi, 200)

fig_cir = go.Figure()

# Unit circle
fig_cir.add_trace(go.Scatter(
    x=np.cos(theta),
    y=np.sin(theta),
    mode="lines",
    line=dict(color="black", dash="dash"),
    showlegend=False
))

# Axes
fig_cir.add_trace(go.Scatter(
    x=[-1, 1], y=[0, 0],
    mode="lines",
    line=dict(color="gray", dash="dash"),
    showlegend=False
))
fig_cir.add_trace(go.Scatter(
    x=[0, 0], y=[-1, 1],
    mode="lines",
    line=dict(color="gray", dash="dash"),
    showlegend=False
))

# Variable vectors (correlations)
for var in quan_cols:
    fig_cir.add_trace(go.Scatter(
        x=[0, Corr_matrix.loc[var, "PC1"]],
        y=[0, Corr_matrix.loc[var, "PC2"]],
        mode="lines+markers+text",
        text=[None, var],
        textposition="top center",
        line=dict(width=2),
        marker=dict(size=6),
        showlegend=False
    ))

fig_cir.update_layout(
    title="PCA Correlation Circle (PC1 vs PC2)",
    xaxis=dict(
        title=f"PC1 ({pca.explained_variance_ratio_[0]*100:.2f}%)",
        range=[-1.2, 1.2],
        zeroline=False
    ),
    yaxis=dict(
        title=f"PC2 ({pca.explained_variance_ratio_[1]*100:.2f}%)",
        range=[-1.2, 1.2],
        zeroline=False
    ),
    width=520,
    height=520
)

fig_cir.show()


In [98]:
scores_df = pd.DataFrame(scores, columns=["PC1", "PC2"], index=X.index)
scores_df.head()

,PC1,PC2
0,-3.067807,0.368798
1,-2.937090,-2.325137
2,-2.527860,-5.009080
3,-2.482625,-0.031152
4,-2.262850,0.451980


# **Machine Learning**

In [99]:
df.columns

Index(['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'price', 'x', 'y',
       'z', 'log_price'],
      dtype='object')

In [100]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.decomposition import PCA

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor

from xgboost import XGBRegressor


from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error


In [101]:
# Features
num_features = ["carat", "x", "y", "z", "depth", "table"]
cat_features = ["cut", "color", "clarity"]

X = df[num_features + cat_features].copy()

# Target (recommended)
y = df["log_price"].copy() 

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

X_train.head()

,carat,x,y,z,depth,table,cut,color,clarity
42144,0.55,5.27,5.29,3.25,61.6,55.0,Ideal,I,VS2
10763,1.01,6.41,6.37,3.96,62.0,58.0,Premium,E,SI1
23057,1.50,7.39,7.44,4.44,59.9,60.0,Premium,H,VS1
11127,0.90,6.18,6.13,3.88,63.0,55.0,Very Good,H,VS1
35702,0.30,4.32,4.27,2.68,62.4,56.0,Ideal,D,VS2


In [102]:
numeric_pca = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("pca", PCA(n_components=2, random_state=42))
])

categorical_ohe = Pipeline(steps=[
    ("ohe", OneHotEncoder(handle_unknown="ignore", drop=None))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num_pca", numeric_pca, num_features),
        ("cat", categorical_ohe, cat_features),
    ],
    remainder="drop"
)


In [103]:
models = {
    "Linear Regression": LinearRegression(),

    "Decision Tree": DecisionTreeRegressor(
        random_state=42
    ),

    "Random Forest": RandomForestRegressor(
        n_estimators=200,
        random_state=42,
        n_jobs=-1
    ),

    "KNN": KNeighborsRegressor(
        n_neighbors=10
    ),

    "Extra Trees": ExtraTreesRegressor(
        n_estimators=200,
        random_state=42,
        n_jobs=-1
    ),

    "Gradient Boosting": GradientBoostingRegressor(
        random_state=42
    ),

    "XGBoost": XGBRegressor(
        n_estimators=300,
        learning_rate=0.05,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8,
        objective="reg:squarederror",
        random_state=42
    ),

    "MLP Regressor": MLPRegressor(
        hidden_layer_sizes=(64, 32),
        max_iter=1000,
        random_state=42
    )
}


In [113]:
import numpy as np
import pandas as pd

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# ---------- Adjusted R2 ----------
def adjusted_r2(r2, n, p):
    return 1 - (1 - r2) * (n - 1) / (n - p - 1)


# ---------- Evaluate on original price scale ----------
def evaluate_backtransformed(model, X_train, X_test, y_train, y_test):
    # Fit model
    model.fit(X_train, y_train)

    # Predict on test set (log scale)
    y_pred_log = model.predict(X_test)

    # Back-transform (to original price)
    y_pred_price = np.expm1(y_pred_log)
    y_true_price = np.expm1(y_test)

    # Metrics on original price scale
    mae = mean_absolute_error(y_true_price, y_pred_price)
    mse = mean_squared_error(y_true_price, y_pred_price)
    rmse = np.sqrt(mse)

    # R² on log scale (standard)
    r2 = r2_score(y_test, y_pred_log)

    # Compute p = number of final features after preprocessing (PCs + one-hot columns)
    Xt = model.named_steps["preprocess"].transform(X_test)
    p = Xt.shape[1]
    n = len(y_test)

    adj_r2 = adjusted_r2(r2, n, p)

    return {
        "MAE_price": mae,
        "MSE_price": mse,
        "RMSE_price": rmse,
        "R2_log": r2,
        "Adj_R2": adj_r2
    }


# ---------- Run all models ----------
results = []

for name, model_ in models.items():
    pipe = Pipeline(steps=[
        ("preprocess", preprocessor),
        ("model", model_)
    ])

    metrics = evaluate_backtransformed(pipe, X_train, X_test, y_train, y_test)
    metrics["Model"] = name
    results.append(metrics)


# ---------- Results table ----------
results_df = pd.DataFrame(results)

# Rename columns to standard names (same format you want)
results_df = results_df.rename(columns={
    "MAE_price": "MAE",
    "MSE_price": "MSE",
    "RMSE_price": "RMSE",
    "R2_log": "R2"
})

results_df = results_df[
    ["Model", "MAE", "MSE", "RMSE", "R2", "Adj_R2"]
].sort_values(by="Adj_R2", ascending=False)

results_df


,Model,MAE,MSE,RMSE,R2,Adj_R2
6,XGBoost,313.108332,3.567943e+05,597.322595,0.987946,0.987921
2,Random Forest,300.326093,3.269138e+05,571.763779,0.986278,0.986250
7,MLP Regressor,335.801309,3.775991e+05,614.490891,0.986210,0.986182
4,Extra Trees,311.181074,3.534995e+05,594.558264,0.984278,0.984246
5,Gradient Boosting,411.044722,6.429555e+05,801.845047,0.982935,0.982900
3,KNN,398.399625,6.161006e+05,784.920769,0.979871,0.979830
1,Decision Tree,383.653772,5.667717e+05,752.842436,0.977125,0.977078
0,Linear Regression,844.603549,1.538905e+07,3922.888186,0.958932,0.958848
